
This code is to mosaic multiple tif raster files in to a single mosaiced map. While mosaicing incase of overlaped layers, we took the maximum value

In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.coords import BoundingBox
import numpy as np
import os

# Define the paths\
input_dir =r"E:\\Global Aggergation Maps\\2000\\District\\Rainfed"
output_dir = r"E:\\Global Aggergation Maps\\2000\\National\\Rainfed"
reference_raster_path = r"D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\AH Grid\\2000\\AEI_00.tif"

# Read the reference raster to use as a template
with rasterio.open(reference_raster_path) as ref_raster:
    ref_bounds = ref_raster.bounds
    ref_crs = ref_raster.crs
    ref_transform = ref_raster.transform
    ref_width = ref_raster.width
    ref_height = ref_raster.height

# List all rasters in the workspace
raster_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]

raster_dict = {}
for raster in raster_files:
    filename = os.path.splitext(os.path.basename(raster))[0]
    crop, unit, ir, planting_month = filename.split("_")
    key = (crop, planting_month)
    if key not in raster_dict:
        raster_dict[key] = [raster]
    else:
        raster_dict[key].append(raster)

for key in raster_dict:
    crop, planting_month = key
    rasters_to_mosaic = raster_dict[key]

    try:
        datasets = [rasterio.open(file) for file in rasters_to_mosaic]

        # Mosaic the datasets
        mosaic, out_transform = merge(datasets, bounds=ref_bounds, method='max')

        # Copy the metadata from the reference dataset to the mosaic
        out_meta = ref_raster.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": ref_height,
            "width": ref_width,
            "transform": ref_transform,
            "crs": ref_crs
        })

        # Create the output path for the mosaic raster
        output_raster = f"{crop}_{planting_month}.tif"
        output_path = os.path.join(output_dir, output_raster)

        # Save the mosaic to the output path
        with rasterio.open(output_path, "w", **out_meta) as dest:
            rounded_mosaic = np.round(mosaic, decimals=2)
            dest.write(rounded_mosaic)

    except Exception as e:
        print(f"Error processing {crop}_{planting_month}: {e}")

    finally:
        # Close all datasets
        for dataset in datasets:
            dataset.close()

This line of code is for preparing Anual harvested area grids by taking a maximum pixe values of all month growing areas

In [ ]:
import rasterio
from rasterio.merge import merge
import numpy as np
import os
import tempfile
import shutil

def clean_raster_data(raster_data, extreme_value_threshold=-1e+38):
    """
    Cleans raster data by replacing extreme negative values, NaNs, and infinites with 0.
    
    Parameters:
    - raster_data: The raster data array.
    - extreme_value_threshold: Values below this threshold are considered missing/extreme.
    
    Returns:
    - Cleaned raster data array.
    """
    raster_data[np.isinf(raster_data)] = 0
    raster_data[np.isnan(raster_data)] = 0
    cleaned_data = np.where(raster_data <= extreme_value_threshold, 0, raster_data)
    return cleaned_data

# Define the input and output directories
input_dir = r"E:\\Global Aggergation Maps\\2010\\National\\Irrigated\\Rice_MAX"
# output_dir = r"E:\\Global Aggergation Maps\\2010\\National\\Irrigated\\Wheat"
# output_dir =  r"E:\\MIRCA-OS Dataset\\Annual Harvested Area Grids\\2010"
output_dir =  r"E:\\MIRCA-OS Dataset\\Maximum Monthly Growing Area Grids\\2010"
os.makedirs(output_dir, exist_ok=True)

# Create a temporary directory for interim files
temp_dir = tempfile.mkdtemp()

try:
    # List all .tif files in the input directory
    raster_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]

    # Clean the data and write to temporary files
    temp_files = []
    for file in raster_files:
        with rasterio.open(file) as src:
            data = src.read(1)  # Read the first band
            cleaned_data = clean_raster_data(data)
            temp_file = os.path.join(temp_dir, os.path.basename(file))
            with rasterio.open(temp_file, 'w', **src.meta) as dst:
                dst.write(cleaned_data, 1)
            temp_files.append(temp_file)

    # Open all cleaned temporary raster datasets
    datasets = [rasterio.open(file) for file in temp_files]

    # Mosaic the datasets using the 'max' method
    mosaic, out_transform = merge(datasets, method='max')

    # Copy the metadata from the first dataset
    out_meta = datasets[0].meta.copy()

    # Update the metadata to reflect the number of layers
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform
    })

    # Output file name
    output_filename = "MIRCA-OS_Rice_2010_ir.tif"
    output_path = os.path.join(output_dir, output_filename)

    # Save the mosaic raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic[0], 1)  # Writing the first index of the mosaic array

    print(f"Mosaic created and saved at {output_path}")

finally:
    # Close all datasets
    for dataset in datasets:
        dataset.close()
    
    # Clean up the temporary files
    shutil.rmtree(temp_dir)


Thsi line of code is to summed up annual maps inacse of multiple seasons (such as Rice and Wheat)

In [ ]:
import rasterio
from rasterio.merge import merge
import numpy as np
import os
import tempfile
import shutil

def clean_raster_data(raster_data, extreme_value_threshold=-1e+38):
    """
    Cleans raster data by replacing extreme negative values, NaNs, and infinites with 0.
    This preparation is crucial for ensuring that non-data values do not skew the summation.
    
    Parameters:
    - raster_data: The raster data array.
    - extreme_value_threshold: Values below this threshold are considered missing/extreme.
    
    Returns:
    - Cleaned raster data array.
    """
    raster_data[np.isinf(raster_data)] = 0  # Replace infinites with zero
    raster_data[np.isnan(raster_data)] = 0  # Replace NaNs with zero
    cleaned_data = np.where(raster_data <= extreme_value_threshold, 0, raster_data)
    return cleaned_data

# Define the input and output directories
input_dir = r"E:\\Global Aggergation Maps\\2010\\National\\Irrigated\\Wheat"
output_dir = r"E:\\MIRCA-OS Dataset\\Annual Harvested Area Grids\\2010"
os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

# Create a temporary directory for interim files
temp_dir = tempfile.mkdtemp()

try:
    # List all .tif files in the input directory
    raster_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]

    # Clean the data and write to temporary files
    temp_files = []
    for file in raster_files:
        with rasterio.open(file) as src:
            data = src.read(1)  # Read the first band
            cleaned_data = clean_raster_data(data)
            temp_file = os.path.join(temp_dir, os.path.basename(file))
            with rasterio.open(temp_file, 'w', **src.meta) as dst:
                dst.write(cleaned_data, 1)
            temp_files.append(temp_file)

    # Open all cleaned temporary raster datasets
    datasets = [rasterio.open(file) for file in temp_files]

    # Mosaic the datasets using the 'sum' method to ensure values are summed in overlaps
    mosaic, out_transform = merge(datasets, method='sum')

    # Copy the metadata from the first dataset
    out_meta = datasets[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform
    })

    # Output file name
    output_filename = "MIRCA-OS_Wheat_2010_ir.tif"
    output_path = os.path.join(output_dir, output_filename)

    # Save the mosaic raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic[0], 1)  # Write the first band of the mosaic

    print(f"Mosaic created and saved at {output_path}")

finally:
    # Close all datasets
    for dataset in datasets:
        dataset.close()
    
    # Clean up the temporary files
    shutil.rmtree(temp_dir)


This line of code is converting multiple tif files in to NETCDF ratsre format

In [ ]:
import os
import re
import rasterio
from rasterio.warp import reproject, Resampling
import xarray as xr
import numpy as np

# Define the folder containing the raster files
raster_folder = "E:\\Global Aggergation Maps\\2000\\National\\Irrigated\\Cassava"

# Reference raster file path
reference_raster_path = r"D:\\PhD_Udel\\MICRA_2015\\DOwnscaling\\AH Grid\\2015\\AEI_15.tif"

# Get a list of all TIFF files in the folder
raster_files = [os.path.join(raster_folder, file) for file in os.listdir(raster_folder) if file.endswith(".tif")]

# Regular expression pattern to extract the month from the filename
pattern = r".*_(\d+)\.tif"

# Read the reference raster to get the desired extent and projection
with rasterio.open(reference_raster_path) as ref:
    ref_transform = ref.transform
    ref_crs = ref.crs
    ref_width = ref.width
    ref_height = ref.height
    ref_bounds = ref.bounds

# Function to convert pixel coordinates to longitude and latitude
def pixel_to_lonlat(transform, width, height):
    x_coords = np.linspace(0.5, width - 0.5, width)
    y_coords = np.linspace(0.5, height - 0.5, height)
    xs, ys = np.meshgrid(x_coords, y_coords)
    lon, lat = rasterio.transform.xy(transform, ys, xs, offset='center')
    return np.array(lon), np.array(lat)

# Convert x and y coordinates to longitude and latitude
lon_coords, lat_coords = pixel_to_lonlat(ref_transform, ref_width, ref_height)

# Create an empty list to store the datasets
datasets = []

# Iterate over each raster file
for raster_file in raster_files:
    # Extract the month from the filename using regex
    match = re.match(pattern, os.path.basename(raster_file))
    if not match:
        print(f"Warning: Unable to extract month from filename: {raster_file}")
        continue

    month = int(match.group(1))

    with rasterio.open(raster_file) as src:
        # Prepare an empty array for the reprojected data
        reprojected_data = np.empty((ref_height, ref_width), dtype=src.read(1).dtype)

        # Reproject the data to match the reference raster
        reproject(
            source=rasterio.band(src, 1),
            destination=reprojected_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=Resampling.nearest,
            dst_width=ref_width,
            dst_height=ref_height
        )

    # Create a dataset using xarray with longitude and latitude
    ds = xr.Dataset(
        data_vars={'harvested_area': (('latitude', 'longitude'), reprojected_data)},
        coords={'longitude': lon_coords[0, :], 'latitude': lat_coords[:, 0]},
        attrs={'crs': ref_crs.to_string()}
    )

    # Add the month coordinate
    ds.coords['month'] = month

    # Append the dataset to the list
    datasets.append(ds)

# Concatenate datasets along the month dimension
merged_ds = xr.concat(datasets, dim='month')

# Write the merged dataset to a NetCDF file
merged_nc = "E:\\MIRCA-OS Dataset\\Monthly Growing Area Grids\\2000\\MIRCA-OS_Cassava_2000_ir.nc"
merged_ds.to_netcdf(merged_nc, format="NETCDF4_CLASSIC", engine="netcdf4")

print(f"Aggregation and reprojection completed. Output saved to: {merged_nc}")
